In [1]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from PIL import Image

import torch
import yaml

import os
import random
from ultralytics.utils.plotting import plot_labels
import pandas as pd
import sys

module_path = os.path.abspath(os.path.join('..'))
print(module_path)
module_path = module_path+'/data_preprocessing'
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)

import visualization_utils as visutils

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
Setup complete ✅ (32 CPUs, 251.6 GB RAM, 0.3/125.8 GB disk)


/vast/palmer/home.grace/eec42/BirdDetector/src
/vast/palmer/home.grace/eec42/BirdDetector/src/data_preprocessing


In [2]:
device = "0" if torch.cuda.is_available() else "cpu"
if device == "0":
    torch.cuda.set_device(0) # Set to your desired GPU number

print(device)

0


In [3]:
# Create a new YOLO model from scratch
#model = YOLO('yolov8n.yaml')

# Load a pretrained YOLO model (recommended for training) - for now, we keep the small version
model = YOLO('yolov8m.pt')

In [4]:
# TRAIN the model on our dataset (data.yml config file) 
model_name = 'pfeifer_newmexico_yolov8m_120epoch_default_batch32_aug90deg0.5flips_patience50'

results = model.train(
   data='data.yaml',
   #imgsz=480,  # we are trying with several img size so we do not precise the size -> will automatically resize all images to 640x640
   epochs=120,
   batch=32,
   #cos_lr=True,
   #dropout=0.3,
   #optimizer='Adam',
   patience=50,
   device=0,
   verbose=True,
   #lr0=0.001,
   #lrf=0.0001,
   degrees=90, fliplr=0.5, flipud=0.5, # augmentation parameters
   name=model_name)

New https://pypi.org/project/ultralytics/8.0.209 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=data.yaml, epochs=120, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=pfeifer_newmexico_yolov8m_120epoch_default_batch32_aug90deg0.5flips_patience50, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buf

THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/train
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/train


train: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/train/labels... 2687 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2687/2687 [00:01<00:00, 2559.64it/s]
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/train/images/global_birds_pfeifer_Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_0.0_0.0_448_448.jpg: 8 duplicate labels removed
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/train/images/global_birds_pfeifer_Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_0.0_2.0_448_448.jpg: 8 duplicate labels removed
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/train/images/global_birds_pfeifer_Rzepecki Islands_middle_2016_Chinstrap_penguins_115_patch_2.0_0.0_448_448.jpg: 8 duplicate labels removed
train: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_back

THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/val
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/val


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/val/labels... 427 images, 0 backgrounds, 0 corrupt: 100%|██████████| 427/427 [00:00<00:00, 2114.89it/s]
val: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/val/images/global_birds_pfeifer_Rzepecki Islands_south_2016_Chinstrap_penguins_103_patch_0.0_0.0_448_448.jpg: 176 duplicate labels removed
val: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/val/images/global_birds_pfeifer_Rzepecki Islands_south_2016_Chinstrap_penguins_103_patch_0.0_2.0_448_448.jpg: 177 duplicate labels removed
val: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/val/images/global_birds_pfeifer_Rzepecki Islands_south_2016_Chinstrap_penguins_103_patch_2.0_0.0_448_448.jpg: 177 duplicate labels removed
val: WARNING ⚠️ /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/val/images/

## Evaluation

In [5]:
# EVALUATE the model's performance on the test set
metrics = model.val(split='test', save_json=True)

# Export the model to ONNX format
#success = model.export(format='onnx')

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/images


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/labels... 896 images, 0 backgrounds, 0 corrupt: 100%|██████████| 896/896 [00:00<00:00, 2749.15it/s]
val: New cache created: /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:10<00:00,  2.61it/s]
                   all        896      19838      0.785       0.69      0.692      0.292
Speed: 0.1ms preprocess, 5.2ms inference, 0.0ms loss, 2.1ms postprocess per image
Saving runs/detect/pfeifer_newmexico_yolov8m_120epoch_default_batch32_aug90deg0.5flips_patience502/predictions.json...
Results saved to runs/detect/pfeifer_newmexico_yolov8m_120epoch_default_batch32_aug90deg0.5flips_patience502


In [6]:
#metrics.box.map    # map50-95
metrics.box.map50  # map50
#metrics.box.map75  # map75
#metrics.box.maps   # a list contains map50-95 of each category

0.6918368282814409

In [7]:
# EVALUATE the model's performance on the test set
metrics = model.val(split='test', save_json=True, iou=0.3, max_det=600)

# Export the model to ONNX format
#success = model.export(format='onnx')

Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)


THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/images


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/labels.cache... 896 images, 0 backgrounds, 0 corrupt: 100%|██████████| 896/896 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:11<00:00,  2.54it/s]
                   all        896      19838      0.778      0.707      0.719       0.28
Speed: 0.1ms preprocess, 5.0ms inference, 0.0ms loss, 1.7ms postprocess per image
Saving runs/detect/pfeifer_newmexico_yolov8m_120epoch_default_batch32_aug90deg0.5flips_patience503/predictions.json...
Results saved to runs/detect/pfeifer_newmexico_yolov8m_120epoch_default_batch32_aug90deg0.5flips_patience503


In [8]:
metrics.box.map50    # map50-95

0.7185976400400489

## Evaluation per dataset

In [25]:
for dataset in datasets:

    # Change test folder
    fname = "data.yaml"
    stream = open(fname, 'r')
    data = yaml.safe_load(stream)
    data['test'] = 'test/' + dataset + '/images/'
    with open(fname, 'w') as yaml_file:
        yaml_file.write( yaml.dump(data, default_flow_style=False))
    
    metrics = model.val(split='test', save_json=True, iou=0.3, max_det=600)
    print(metrics.box.map50)


Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)


THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/global_birds_pfeifer/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/global_birds_pfeifer/images


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/global_birds_pfeifer/labels... 720 images, 0 backgrounds, 0 corrupt: 100%|██████████| 720/720 [00:00<00:00, 2979.78it/s]
val: New cache created: /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/global_birds_pfeifer/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:07<00:00,  5.66it/s]
                   all        720      15807      0.828      0.811      0.794      0.322
Speed: 0.1ms preprocess, 5.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Saving runs/detect/val/predictions.json...
Results saved to runs/detect/val
Ultralytics YOLOv8.0.199 🚀 Python-3.11.6 torch-2.1.0+cu121 CUDA:0 (NVIDIA RTX A5000, 24248MiB)


0.7940600515389735
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/global_birds_newmexico/images
THE PATH IS, PATH :   /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/global_birds_newmexico/images


val: Scanning /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/global_birds_newmexico/labels... 176 images, 0 backgrounds, 0 corrupt: 100%|██████████| 176/176 [00:00<00:00, 1656.68it/s]
val: New cache created: /gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/global_birds_newmexico/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:02<00:00,  3.68it/s]
                   all        176       4031      0.788        0.5      0.581      0.205
Speed: 0.1ms preprocess, 5.0ms inference, 0.0ms loss, 4.9ms postprocess per image
Saving runs/detect/val2/predictions.json...
Results saved to runs/detect/val2


0.5813462064245143


## Visualization

In [20]:
model_name = 'pfeifer_newmexico_yolov8m_120epoch_default_batch32_aug90deg0.5flips_patience50'
model = YOLO('runs/detect/' + model_name + '/weights/best.pt')
datasets = ['global_birds_pfeifer', 'global_birds_newmexico']

In [23]:
# Select randomly 10 images from the test dataset
selected_img = []
for subdataset in datasets:
    selected_img.extend(random.choices(os.listdir('/gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/' + subdataset + '/images/'), k=8))

results = model.predict(
        #model = 'runs/detect/pfeifer_yolov8n_70epoch_default_batch32_dropout0.3',
        source = [os.path.join('/gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/images/', img) for img in selected_img],
        conf = 0.2, 
        iou = 0.3,
        show=False,
        save=False
    )


0: 640x640 10 birds, 1: 640x640 5 birds, 2: 640x640 16 birds, 3: 640x640 54 birds, 4: 640x640 1 bird, 5: 640x640 16 birds, 6: 640x640 1 bird, 7: 640x640 2 birds, 8: 640x640 8 birds, 9: 640x640 26 birds, 10: 640x640 18 birds, 11: 640x640 19 birds, 12: 640x640 9 birds, 13: 640x640 10 birds, 14: 640x640 18 birds, 15: 640x640 10 birds, 104.3ms
Speed: 2.5ms preprocess, 6.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)


In [24]:
for img, result in zip(selected_img, results):

    detection_boxes = []
    save_path = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/prediction_' + os.path.basename(result.path).split('.png')[0] + '.jpg'
    for detect in range(len(result.boxes.cls)):
        det = {}
        det['conf'] = result.boxes.conf[detect].cpu()
        det['category'] = result.boxes.cls[detect].cpu()
        coords = result.boxes.xywhn[detect].cpu()
        det['bbox'] = [coords[0]-coords[2]/2, coords[1]-coords[3]/2, coords[2], coords[3]]
        detection_boxes.append(det)
        
    im_path = os.path.join('/gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/images/', img)
    visutils.draw_bounding_boxes_on_file(im_path, save_path, detection_boxes,
                                    confidence_threshold=0.0, detector_label_map=None,
                                    thickness=1,expansion=0, colormap=['Red'])

    selected_label = '/gpfs/gibbs/project/jetz/eec42/data/baseline1_pfeifer_newmexico_no_background/test/labels/' + os.path.basename(result.path).split('.jpg')[0] + '.txt'
    detection_boxes = []
    df = pd.read_csv(selected_label, sep='\t', header=None, index_col=False)
    for irow, row in df.iterrows():  
        det = {}
        det['conf'] = None
        det['category'] = row[0]
        det['bbox'] = [row[1]-row[3]/2, row[2]-row[4]/2, row[3], row[4]]
        detection_boxes.append(det)
    
    # Draw annotations
    save_path2 = '/vast/palmer/home.grace/eec42/BirdDetector/src/model/runs/detect/' + model_name + '/prediction_label_' + os.path.basename(result.path).split('.png')[0] + '.jpg'
    visutils.draw_bounding_boxes_on_file(save_path, save_path2, detection_boxes,
                                    confidence_threshold=0.0, detector_label_map=None,
                                    thickness=1,expansion=0, colormap=['SpringGreen'])
        